# Daily Indicators per Stock

In [1]:
# Import libraries and dependencies
import pandas as pd
import numpy as np
%matplotlib inline



## Read in Fundamental Indicators

In [63]:
free_cash_flow_equity_path = 'Resources/s_FreeCashFlowEquity.csv'
free_cash_flow_equity_df = pd.read_csv(free_cash_flow_equity_path)

gross_margin_path = 'Resources/s_GrossMargin.csv'
gross_margin_df = pd.read_csv(gross_margin_path)

market_cap_path = 'Resources/s_MarketCap.csv'
market_cap_df = pd.read_csv(market_cap_path)

net_income_margin_path = 'Resources/s_NetIncomeMargin.csv'
net_income_margin_df = pd.read_csv(net_income_margin_path)

pb_path = 'Resources/s_PB.csv'
pb_df = pd.read_csv(pb_path)

pe_path = 'Resources/s_PE.csv'
pe_df = pd.read_csv(pe_path)

px_path = 'Resources/s_PX.csv'
px_df = pd.read_csv(px_path)

roa_path = 'Resources/s_ROA.csv'
roa_df = pd.read_csv(roa_path)

roe_path = 'Resources/s_ROE.csv'
roe_df = pd.read_csv(roe_path)

universe_path = 'Resources/s_universe.csv'
universe_df = pd.read_csv(universe_path)

C:\Users\ekuo651\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [38]:
index_dates_df = pd.read_csv(px_path)

In [13]:
fundamental_indicators = [free_cash_flow_equity_df,
                          gross_margin_df,
                          market_cap_df,
                          net_income_margin_df,
                          pb_df,
                          pe_df,
                          px_df,
                          roa_df,
                          roe_df]
sectors_df = universe_df

In [66]:
index_dates = list(px_df['Date'])
index_dates[:3]

['1/1/1998', '1/2/1998', '1/5/1998']

In [18]:
[print(i.shape) for i in fundamental_indicators]

(5724, 505)
(5724, 505)
(5724, 505)
(5724, 505)
(263, 505)
(5724, 505)
(5724, 505)
(5724, 505)
(263, 505)


[None, None, None, None, None, None, None, None, None]

In [67]:
def drop_date(df):
    return df.drop(columns='Date', inplace=True)

[drop_date(i) for i in fundamental_indicators]

In [17]:
stocks = list(px_df.column)
stocks[:3]

['ILMN UW Equity', 'EIX UN Equity', 'GPS UN Equity']

In [105]:
stocks[76:80]

['BDX UN Equity', 'BRK/B UN Equity', 'BBY UN Equity', 'HRB UN Equity']

['df_ILMN UW Equity',
 'df_EIX UN Equity',
 'df_GPS UN Equity',
 'df_ES UN Equity',
 'df_NLOK UW Equity']

## Make DataFrames for Individual Stocks

In [45]:
column_names = ['free_cash_flow_equity',
               'gross_margin',
               'market_cap',
               'net_income_margin',
               'pe',
               'px',
               'roa']


def make_df(stock):
    new_df = pd.concat([free_cash_flow_equity_df[stock],
                    gross_margin_df[stock], 
                    market_cap_df[stock],
                    net_income_margin_df[stock],
                    pe_df[stock],
                    px_df[stock],
                    roa_df[stock]],
                   axis='columns')
    new_df.columns = column_names
    new_df.index = index_dates
    return new_df


In [82]:
# dropna(axis=1,how='all') drops columns that are completely null
# dropna() will drop rows with any remaining nulls, essentially the null period before fundamentals were available

make_df(stocks[450]).ffill().dropna(axis=1,how='all').dropna()


,free_cash_flow_equity,market_cap,net_income_margin,pe,px,roa
7/1/1998,49.727,2906.6080,19.1075,19.6910,0.8984,15.3831
7/2/1998,49.727,2221.1815,19.1075,15.0475,0.9115,15.3831
7/3/1998,49.727,2266.0527,19.1075,15.3409,0.9115,15.3831
7/6/1998,-28.386,2007.6943,16.3669,13.7649,0.9531,15.1054
7/7/1998,-28.386,2251.8846,16.3669,15.4391,0.9609,15.1054
7/8/1998,-28.386,2469.9365,16.3669,16.9476,0.9740,15.1054
7/9/1998,100.062,2614.0023,20.5305,18.3556,0.9896,14.4705
7/10/1998,100.062,2555.7098,20.5305,17.9463,0.9948,14.4705
7/13/1998,100.062,2916.5513,20.5305,20.2737,1.0026,14.4705
7/14/1998,113.933,2970.3902,18.7846,21.3987,0.9844,13.8553


## Add Technical Indicators

In [95]:
# Preset Parameters
window_year=252
window_month=21
window_quarter=63

In [98]:
def add_technical_indicators(stock):

    stock_df = make_df(stock).ffill().dropna(axis=1,how='all').dropna()
    stock_df['Price Momentum 12M'] = stock_df.px/stock_df.px.shift(window_year)-1
    stock_df['Price Momentum 12M-1M'] = stock_df.px.shift(window_month)/stock_df.px.shift(window_year)-1
    stock_df['Price to 52W High'] = stock_df.px/stock_df.px.rolling(window_year).max()
    stock_df['Price to 52W Low'] = stock_df.px/stock_df.px.rolling(window_year).min()
    stock_df['Return 1D'] = stock_df.px/stock_df.px.shift(1)-1
    stock_df['Return 1M'] = stock_df.px/stock_df.px.shift(window_month)-1
    stock_df['Return 1Q'] = stock_df.px/stock_df.px.shift(window_quarter)-1
#     stock_df['Return 5Y'] = stock_df.px/stock_df.px.shift(window_year*5)-1
    stock_df['Max Daily Return over 1M'] = stock_df['Return 1D'].rolling(window_month).max()
    stock_df['Realized Volatility 1M'] = stock_df['Return 1D'].rolling(window_month).std()
    stock_df['Realized Volatility 3M'] = stock_df['Return 1D'].rolling(window_month*3).std()
    stock_df.dropna(inplace=True)
    return stock_df
    

In [100]:
add_technical_indicators(stocks[2])

,free_cash_flow_equity,gross_margin,market_cap,net_income_margin,pe,px,roa,Price Momentum 12M,Price Momentum 12M-1M,Price to 52W High,Price to 52W Low,Return 1D,Return 1M,Return 1Q,Max Daily Return over 1M,Realized Volatility 1M,Realized Volatility 3M
6/11/1999,-254.0,38.9945,6200.5029,4.9775,7.8755,29.5437,7.2047,0.589396,0.518248,0.847907,2.034354,0.003863,0.046862,-0.049785,0.051896,0.022758,0.028058
6/14/1999,-254.0,38.9945,6200.5029,4.9775,7.8755,28.8044,7.2047,0.552789,0.535140,0.826689,1.983446,-0.025024,0.011497,-0.065878,0.051896,0.023466,0.028213
6/15/1999,-254.0,38.9945,6200.5029,4.9775,7.8755,28.8897,7.2047,0.565362,0.504503,0.829137,1.989320,0.002961,0.040452,-0.065696,0.051896,0.022713,0.028214
6/16/1999,-254.0,38.9945,6200.5029,4.9775,7.8755,29.8849,7.2047,0.582007,0.475885,0.857699,2.057849,0.034448,0.071904,-0.025449,0.051896,0.023784,0.028540
6/17/1999,-254.0,38.9945,6200.5029,4.9775,7.8755,30.3399,7.2047,0.627267,0.480099,0.870758,2.089179,0.015225,0.099431,0.008074,0.051896,0.023695,0.028503
6/18/1999,-254.0,38.9945,6200.5029,4.9775,7.8755,31.5057,7.2047,0.654555,0.482066,0.904216,2.169455,0.038425,0.116384,0.065934,0.051896,0.024525,0.028797
6/21/1999,-254.0,38.9945,6200.5029,4.9775,7.8755,32.3019,7.2047,0.679687,0.455668,0.927067,2.224281,0.025272,0.153895,0.127564,0.051896,0.024679,0.028652
6/22/1999,-254.0,38.9945,6200.5029,4.9775,7.8755,31.6905,7.2047,0.647895,0.468969,0.909520,2.182181,-0.018928,0.121803,0.042125,0.051896,0.025316,0.027750
6/23/1999,-254.0,38.9945,6200.5029,4.9775,7.8755,31.7332,7.2047,0.599778,0.376880,0.910746,2.185121,0.001347,0.161886,0.053374,0.051896,0.023728,0.027718
6/24/1999,-254.0,38.9945,6200.5029,4.9775,7.8755,31.7332,7.2047,0.624561,0.411301,0.910746,2.185121,0.000000,0.151109,0.034824,0.051896,0.023778,0.027635


## Export Daily Fundamental & Technical Indicator DataFrames

In [109]:
def export_csv(stock):
    df = add_technical_indicators(stock)
    name = str(stock).replace('/','')
    filename = 'Daily_Stocks_Metrics/'+name+'.csv'   
    df.to_csv(filename)

In [111]:
[export_csv(stock) for stock in stocks]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [112]:
def csv_name(stock):
    name = str(stock).replace('/','')
    filename = 'Daily_Stocks_Metrics/'+name+'.csv'
    return filename

csv_names = [csv_name(stock) for stock in stocks]

csv_names[5:10]

['Daily_Stocks_Metrics/AET UN Equity.csv',
 'Daily_Stocks_Metrics/MCO UN Equity.csv',
 'Daily_Stocks_Metrics/RMD UN Equity.csv',
 'Daily_Stocks_Metrics/LYB UN Equity.csv',
 'Daily_Stocks_Metrics/AXP UN Equity.csv']

In [115]:
csv_names_df = pd.DataFrame(csv_names)
csv_names_df.head()

,0
0,Daily_Stocks_Metrics/ILMN UW Equity.csv
1,Daily_Stocks_Metrics/EIX UN Equity.csv
2,Daily_Stocks_Metrics/GPS UN Equity.csv
3,Daily_Stocks_Metrics/ES UN Equity.csv
4,Daily_Stocks_Metrics/NLOK UW Equity.csv


In [116]:
csv_names_df.to_csv('Daily_Stocks_Metrics/csv_names_list.csv',index=False)